In [2]:
import pandas as pd

f=open('d:/data/text/news1.txt', 'r', encoding='utf-8')
lines=f.readlines()
f.close()

In [3]:
import re
from konlpy.tag import Hannanum

han=Hannanum()
word_list=[]
for i in range(len(lines)):
    sentence=re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', lines[i])
    # 명사만 추출
    a=sentence.strip()
    n=han.nouns(a)
    n2=[x for x in n if len(x)>1]
    if len(n2)>0:
        word_list.append(n2)

In [4]:
# TF-IDF 행렬을 만들기 위해 다시 역토큰화
detokenized_doc=[]
for i in range(len(word_list)):
    t=' '.join(word_list[i])
    detokenized_doc.append(t)

detokenized_doc

['치맥전쟁 제너시스BBQ 신제품 레트 감성 트렌디한 세대 겨냥 공략 치킨업계 처음 수제 맥주 맥주 신제품 개발 기선제압',
 '오후 가을겨울 성수 겨냥 신제품 익숙 비주얼 옛날통닭 재해석 마리 세트 갈릭 와사비 구성 한국인 입맛 마늘 시작 코끝 와사비 마무리',
 '베어무 순간 맥주 바삭한 껍질 속살 육즙 입안 맥주 판매 모금 완벽 궁합 표현',
 '가격 옛날 통닭 옛날 물가 반영 기분 소비자 이날 치킨 매장 옛날통닭 가격 마리 가격 마리 가구 마리 판매 설명 가격',
 '맞은닭 특제 간장소스 갈릭 후레이크 쌓아올린 제품 처음 치즈 생각 가루 갈릭 후레이크 베어물자 향긋 갈릭향 짭짤 소스',
 '맞은닭 안주 사람 치맥 메뉴 단짠 정석 계속 맥주 가격',
 '마지막 까먹 강렬한 비주얼 자랑 비주얼 플레이팅된 감귤칩 선명 셋팅된 백년 소스 사진 충동 메뉴',
 '메뉴 넓적다리살 오징어 먹물 세대 취향 저격 출시 제주맥주 협업 치어스 메뉴 제주 감성 치킨 현무암 연상 감귤칩 제주 감귤칩 백년 제주 재료',
 '강렬한 까먹 비주얼 바삭 소리 특유 황금올리브 강렬 튀김 오징어 먹물 느끼긴 백년 소스 풍부',
 '진입장벽 비주얼 합격점 가격 재미 감안 적절 이날 까먹 메신저 프로그램 친구들 대부분 신기 반응 친구 연인 치킨 기회 까먹 치킨 도전',
 '소비자 다양한 니즈 충족 신제품 개발 황금올리브 스테디셀러 탄생 관심 이유',
 '교촌 참전 치맥전쟁 업계 관심사 옥토버페스트 운영사인 마이크로브루어리코리아 수제맥주 개발 대규모 수제맥주 공장 건립 추진 교촌 수제맥주 브랜드 문베어브루잉 운영 인덜지 인수 업계 치맥전쟁 예고']

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer()
X=vectorizer.fit_transform(detokenized_doc)
X.shape # TF-IDF 행렬의 크기 확인

(12, 150)

In [6]:
from sklearn.decomposition import TruncatedSVD
# n_components : 토픽 수
svd_model=TruncatedSVD(n_components=17)
svd_model.fit(X)
len(svd_model.components_)

12

In [7]:
import numpy as np
# 토픽수 × 단어수
np.shape(svd_model.components_)

(12, 150)

In [8]:
svd_model.components_

array([[ 0.227226  ,  0.03969549,  0.02596375, ...,  0.0537854 ,
         0.1215905 ,  0.0519275 ],
       [ 0.1592092 ,  0.03560502,  0.01656862, ..., -0.02657205,
        -0.02334556,  0.03313724],
       [ 0.43180992,  0.10168748, -0.01330248, ..., -0.0275865 ,
        -0.12978825, -0.02660495],
       ...,
       [-0.08902462, -0.08598913, -0.0592772 , ..., -0.081573  ,
        -0.06345342, -0.11855441],
       [ 0.11181658,  0.0482677 , -0.00869111, ..., -0.0086737 ,
         0.06362435, -0.01738222],
       [ 0.14117363,  0.09295341,  0.01232701, ..., -0.04448599,
        -0.11597647,  0.02465403]])

In [9]:
# 단어 집합
terms=vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d : " % (idx+1),
        [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model.components_, terms)
# 각 토픽의 핵심 키워드 추출

Topic 1 :  [('비주얼', 0.28391), ('까먹', 0.24848), ('가격', 0.22723), ('메뉴', 0.22562), ('백년', 0.19862)]
Topic 2 :  [('맥주', 0.37893), ('신제품', 0.24699), ('가격', 0.15921), ('개발', 0.15284), ('치맥전쟁', 0.1387)]
Topic 3 :  [('가격', 0.43181), ('마리', 0.27331), ('옛날', 0.20337), ('치킨', 0.1823), ('이날', 0.1587)]
Topic 4 :  [('와사비', 0.22554), ('마리', 0.17892), ('신제품', 0.1703), ('갈릭', 0.15967), ('소비자', 0.13243)]
Topic 5 :  [('갈릭', 0.33195), ('후레이크', 0.32099), ('맞은닭', 0.17448), ('가루', 0.16049), ('특제', 0.16049)]
Topic 6 :  [('수제맥주', 0.41762), ('교촌', 0.27841), ('업계', 0.27841), ('치맥전쟁', 0.22765), ('예고', 0.13921)]
Topic 7 :  [('제주', 0.365), ('메뉴', 0.2266), ('감귤칩', 0.18139), ('치어스', 0.12167), ('출시', 0.12167)]
Topic 8 :  [('와사비', 0.26678), ('겨냥', 0.14361), ('오후', 0.13339), ('코끝', 0.13339), ('익숙', 0.13339)]
Topic 9 :  [('치킨', 0.23462), ('제주', 0.19426), ('메신저', 0.13938), ('친구들', 0.13938), ('적절', 0.13938)]
Topic 10 :  [('계속', 0.16483), ('안주', 0.16483), ('정석', 0.16483), ('치맥', 0.16483), ('단짠', 0.16483)]
Topic 11 :  [('소리